In [28]:
import pandas as pd
from flashtext import KeywordProcessor
import os
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os
import io
import PyPDF2
from PyPDF2 import PdfFileReader
import re
from datetime import datetime
import time
import math
import fitz
import time
from docx import Document
from win32com import client as wc
import docx2txt
from langdetect import detect
from googletrans import Translator

In [2]:
documenttitlef = {'SOW': 'Contract', 'Contract': 'Contract', 'HPE Customer Terms - Portfolio': 'Contract', 'Agreement': 'Contract', 'Addenda': 'Supportive Documents', 'Amendment': 'Supportive Documents', 'Exhibit': 'Supportive Documents', 'Emails': 'Supportive Documents', 'Addendum': 'Supportive Documents', 'PA Request form': 'Supportive Documents', 'Coversheet': 'Supportive Documents', 'Checklist': 'Supportive Documents', 'Customer Quote': 'Supportive Documents', 'Extension': 'Supportive Documents', 'Annexure': 'Supportive Documents', 'Schedule': 'Supportive Documents','Other Documents': 'Supportive Documents'}

In [3]:
def language_criteria():
    sheet_list=['English','Dutch','Spanish','Czech','French','Italian','German','Danish','Portuguese']
    lang_code=['en','nl','es','cs','fr','it','de','da','pt']
    base_dict={name:{} for name in lang_code}

    i=0
    for name in sheet_list:
        df = pd.read_excel("Language Translation.xlsx",sheet_name=name)
        print(name)
        res = []
        keywordsdict = {}
        documenttitledict= {}
        for row in range(0,len(df)):
            word = df['Translation'][row]
            words = word.split(',')
            record = df['Record Type'][row]
            if record not in keywordsdict:
                keywordsdict[record] = []
            for word in words:
                keywordsdict[record].append(word)
#         print(keywordsdict)
        title = df['Document Title'][row]
        documenttitledict[record] = title
    #     base_dict[name]=keywordsdict
        base_dict[lang_code[i]] = keywordsdict
        i+=1
    final_dict={}
    for i in base_dict.keys():
        for j in base_dict[i]:
            if j in final_dict:
                final_dict[j].extend(base_dict[i][j])
            else:
                final_dict.update({j:base_dict[i][j]})
    for key in final_dict.keys():
        re = final_dict[key]
        re = list(set(re))
        final_dict[key] = re
    return final_dict

In [4]:
final_dict = language_criteria()
documenttitlef

English
Dutch
Spanish
Czech
French
Italian
German
Danish
Portuguese


{'SOW': 'Contract',
 'Contract': 'Contract',
 'HPE Customer Terms - Portfolio': 'Contract',
 'Agreement': 'Contract',
 'Addenda': 'Supportive Documents',
 'Amendment': 'Supportive Documents',
 'Exhibit': 'Supportive Documents',
 'Emails': 'Supportive Documents',
 'Addendum': 'Supportive Documents',
 'PA Request form': 'Supportive Documents',
 'Coversheet': 'Supportive Documents',
 'Checklist': 'Supportive Documents',
 'Customer Quote': 'Supportive Documents',
 'Extension': 'Supportive Documents',
 'Annexure': 'Supportive Documents',
 'Schedule': 'Supportive Documents',
 'Other Documents': 'Supportive Documents'}

In [5]:
def ocr(pdfpath):
#     try:
        pdfFileObj = PyPDF2.PdfFileReader(pdfpath, 'rb')
        a = pdfFileObj.getNumPages()
        doc = fitz.open(pdfpath)
        for i in range(1):
                page = doc.loadPage(i)  # number of page
                pix = page.getPixmap()
                #output = "out_"+str(cnt)+".png"
                output = pdfpath[:-4]+"_"+str(i)+".jpg"
                #print(output)
                pix.writePNG(output)
                im = Image.open(output)
                im.save(output, dpi=(600,600))


        # output = pdfpath[:-4]+"_"+str(1)+".jpg"
        # pix.writePNG(output)
        # im = Image.open(output)
        # im.save(output, dpi=(600,600))


        pytesseract.pytesseract.tesseract_cmd ='C:\\Users\\rnavn\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'
        text = str(((pytesseract.image_to_string(Image.open(output)))))
        print(text)
        return text
#     except OSError:
#         return "fnf err"
#     except:
#         return "ecr err"

In [6]:
def readdoc(filepath,file_extension):
#     try:
        if file_extension == "docx":
            my_text = docx2txt.process(filepath)
            print(my_text)
            return my_text

        #     file_path = r'C:\Users\rnavn\Contact Migration\wer.doc'
        else:
            word = wc.Dispatch("Word.Application")
            doc = word.Documents.Open(filepath)
        #     doc.SaveAs(r'C:\Users\rnavn\Contact Migration\wer2.docx', 12) #12 stands for docx
            filepath = filepath + "x"
            doc.SaveAs(filepath,12)
            doc.Close()
            print(filepath)
            word.Quit()
            my_text = docx2txt.process(filepath)
            print(my_text)
            return my_text
#     except OSError:
#         return "fnf err"
#     except:
#         return "ecr err"

In [7]:
def readex(filepath):
#     try:
        sp = pd.read_excel(filepath)
        sp = sp.fillna('-')
        # arr = [x for x in range(0,len(sp.columns))]
        # p = pd.DataFrame(sp, columns = arr)
        col = list(sp.columns)
        sp
        finalstr = ""
        sp.columns
        for i in range(len(sp)):
            for j in range(len(col)):
                finalstr += str(sp[col[j]][i]) + " "
        print(finalstr)
        # res = findrecord(finalstr)
        return finalstr
#     except OSError:
#         return "fnf err"
#     except:
#         return "ecr err"

In [8]:
def findrecord(textextract):
    sample = textextract
    kp = KeywordProcessor()
    kp.add_keywords_from_dict(final_dict)
    res = kp.extract_keywords(sample)
    staus = ""
    
    priority = ['Extension','Addenda','Amendment','Addendum','Exhibit','Annexure','Coversheet','Checklist','Customer Quote','PA Request form','Contract','Agreement','Schedule','HPE Customer Terms - Portfolio','SOW','Emails','Other Documents']
    # test_list.sort(key = lambda i: sort_order.index(i[0]))
    orderres = list(set(res))
    
    print(orderres)
    if len(orderres) == 0:
        recordtype = ""
        documenttype = ""
        status = "Unavailable"
    else:
        orderres.sort(key = lambda i: priority.index(i))
        print(orderres)
        recordtype = orderres[0]
        documenttype = documenttitlef[orderres[0]]
        status = "Found"
#     res = []
#     res.append([filename,recordtype,documenttype])
#     if len(orderres) >= 2:
#         recordtype = recordtype + "More than one"
    ans = [recordtype,documenttype,status]
    return ans

In [9]:
def readdata(finalpath,file_extension):
    docs = set(['doc','docx','DOC','DOCX'])
    pdfs = set(['pdf','PDF'])
    exc = set(['xlsx','xls','XLS','XLSX'])
    other = set(['zip','ZIP','xlsb','xlsm','XLSM','XLSB'])
    if file_extension in pdfs:
        textextract = ocr(finalpath)
    elif file_extension in docs:
        textextract = readdoc(finalpath,file_extension)
    elif file_extension in exc:
        textextract = readex(finalpath)
    else:
        textextract = "None"
    return textextract

In [11]:
ft = pd.read_excel('iCertisPythonReportv3.xlsx')
ft

,Unnamed: 0,PA No,File Name,Path,Customer Name,Record Type,Document Title,PA No Availability Status,Record Type Availability Status,File extension
0,0,BM2AR,BM2AR_ LOE Econom Lease SA_2018-26-06-18 until...,/EMEAContracts,NaN,NaN,NaN,Active,Unavailable,pdf
1,1,NaN,EMEA CC VD DATA EDITOR 28.09.2015_Backup.mdb,/EMEAContracts,NaN,NaN,NaN,Manual Check,NaN,mdb
2,2,NaN,Approval PA SW305 -ABB.msg,/EMEAContracts/Contracts/ABB - SW305/2007,NaN,Email,Supporting Documents,Inactive,Available,msg
3,3,D26DC,FW Global Frame Agreement DaimlerChrysler.msg,/EMEAContracts/Contracts/Daimler - D26DC/2006 ...,NaN,Email,Supporting Documents,Active,Available,msg
4,4,JM2AT,FW Vetragscheck HP.msg,/EMEAContracts/GWE South/GWE South/Austria - J...,NaN,Email,Supporting Documents,Active,Available,msg
...,...,...,...,...,...,...,...,...,...,...
44497,44497,NaN,CRL DG Stores.pdf,/EMEAContracts/MEMA/South Africa/VSI/XMD47 - S...,NaN,NaN,NaN,Inactive,NaN,pdf
44498,44498,NaN,form.xls,/EMEAContracts/MEMA/South Africa/VSI/XMD47 - S...,NaN,NaN,NaN,Inactive,NaN,xls
44499,44499,NaN,preliminary PA#.msg,/EMEAContracts/MEMA/South Africa/VSI/XMD47 - S...,AF,Email,Supporting Documents,Inactive,Available,msg
44500,44500,NaN,validation.msg,/EMEAContracts/MEMA/South Africa/VSI/XMD47 - S...,AF,Email,Supporting Documents,Inactive,Available,msg


In [23]:
fnft = ft['Record Type Availability Status'] == "Unavailable"
# fileerr = out[7] == "Unavailable"
fra = ft[fnft]
fra.style
m = fra['Unnamed: 0']
ind = []
for i in m.index:
    ind.append(i)
ind
# print(len(ind))

[0,
 9,
 150,
 178,
 182,
 236,
 250,
 258,
 260,
 261,
 275,
 280,
 313,
 321,
 323,
 397,
 556,
 564,
 646,
 750,
 808,
 868,
 956,
 963,
 1024,
 1115,
 1183,
 1187,
 1202,
 1360,
 1402,
 1424,
 1429,
 1462,
 1482,
 1490,
 1569,
 1698,
 1699,
 1713,
 1724,
 1753,
 1754,
 1767,
 1768,
 1780,
 1791,
 1802,
 1813,
 1824,
 1826,
 1846,
 1879,
 1891,
 1906,
 1979,
 2013,
 2024,
 2034,
 2044,
 2045,
 2057,
 2121,
 2138,
 2147,
 2306,
 2380,
 2391,
 2458,
 2480,
 2646,
 2928,
 3133,
 3134,
 3265,
 3274,
 3275,
 3276,
 3286,
 3311,
 3338,
 3341,
 3349,
 3485,
 3486,
 3536,
 3759,
 3828,
 3882,
 3901,
 3916,
 3994,
 4014,
 4054,
 4056,
 4060,
 4163,
 4272,
 4437,
 4443,
 4522,
 4532,
 4582,
 4610,
 4614,
 4651,
 4653,
 4696,
 4795,
 4810,
 4820,
 4841,
 4867,
 4903,
 4933,
 4981,
 5107,
 5123,
 5141,
 5152,
 5166,
 5180,
 5194,
 5237,
 5403,
 5532,
 7511,
 7634,
 8035,
 8046,
 8258,
 8592,
 8617,
 8624,
 8629,
 8654,
 8664,
 8705,
 8722,
 8727,
 8775,
 8816,
 8822,
 8847,
 8852,
 8853,
 8868,

In [34]:
new = 0
ocrsucess = 0
noteng = 0
errocr = 0
result = []
fnfocr = 0
ecrocr = 0
bt = set([-1,26402])
# i = 39420
for i in fra.index:
    print(i)
    file_paths=[]
    pa_no = ft['PA No'][i]
    file_name = ft['File Name'][i]
    path = ft['Path'][i]
    customer_name = ft['Customer Name'][i]
    record_type = ft['Record Type'][i]
    document_title = ft['Document Title'][i]
    pa_availability = ft['PA No Availability Status'][i]
    record_availability = ft['Record Type Availability Status'][i]
    file_extension = ft['File extension'][i]
    ex = set(['pdf','PDF','doc','DOC','docx','DOCX','xls','XLS','xlsx','XLSX','XLSM','xlsm','XLSB','xlsb','ZIP','zip'])
    #print(file_name)
    if record_availability == "Unavailable" and pa_availability == "Active" and i not in bt:
        if file_extension not in ex:
            record_availibility = "Available"
            record_type = "Other Documents"
            document_title = "Supporting Documents"
#             continue
        else:
            filepath = path.replace("/","\\")
            ap = "C:\\Users\\rnavn\\OneDrive - Hewlett Packard Enterprise"
            finalpath = ap + filepath + "\\" + file_name
            #print(finalpath)
            #print(type(filepath))
    #         file_paths.append(filepath)
    #         if filepath in folder_list:
                #print("HERE")
                #print("FINALPATH",finalpath)
            textextract = readdata(finalpath,file_extension)
                #print(textextract)
            if textextract == "None":
                    new += 1
            elif textextract == "fnf err":
                    record_availability = "File Not Found Error"
                    fnfocr += 1
            elif textextract == "ecr err":
                    record_availability = "Encryption Error"        

            else:
    #                 keywordsdict= language_criteria()
                    translator = Translator()
                    t = translator.translate(textextract)
                    transtext = t.text
                    finalans = findrecord(transtext)
                    record_availability = finalans[2]
                    record_type = finalans[0]
                    document_title = finalans[1]
                    ocrsucess += 1
                   
    print([pa_no,file_name,path,customer_name,record_type,document_title,pa_availability,record_availability,file_extension])
    result.append([pa_no,file_name,path,customer_name,record_type,document_title,pa_availability,record_availability,file_extension])


0
 

From: Ezenocomasee SA te
‘Nezon Pos Lnuvnseteonon 51080
ecogwvenTene

be.ciuw
TVA ans 321702

 

 

 

‘We we you erence he request fo above mertonad Custom om HP Wo aow ufo he sle upon tein he
Iatee HP pred the Castano gay He or dee at Cosa acs on MP une! be eeeren utean HP a a
stone rhe act si SMZAR and ne (te Const BUZAR agree

 

“Tit cont tat we age yt MP. win they (30) Says of te ate of H's mnie. any ince at HP ses to uso
{Galen ote Apeenat ject the HP pesuls beng uy Geet to Casta and conapanang Pest of every (POO)
‘Song gna bythe Caster" unaecaer ponte wth scopy ony POD Pat we cd atin ‘eae cnn
{Siva to he Customer mP may sou tects spon dee isthe sip” Coser stress a8 specs m the Scape
‘Gviomars exer. The sbore apy pease of ay agrewrat toe convey taweer HP and Clome ang epatns of ty
‘gis an eagitons appeals teens the Leasing Corgay aod Casome The tte product ree by HE vl Wh
{aye ane wl paste Leasing Congaty any ween san oye cel Rawat purases he Unite Sates whee
‘pasos son sev

['Contract']
['Contract']
['JM3MG', 'JM3MG.pdf', '/EMEAContracts/GWE South/GWE South/Austria - J/Standard - Custom/JM3MG - MAGNA STEYR Fahrzeugtechnik AG & Co KG', 'MAGNA STEYR Fahrzeugtechnik AG & Co KG', 'Contract', 'Contract', 'Active', 'Found', 'pdf']
182
sommoscSRETSEN Taaes cnn nomamnar

Debora way
ag f
4g vonensiumipy sas

ae om Pe"

pone be omens

ig “sunok AyasaDus

‘2 hve ouonessodue 1 Ayauapu Aue oy edo Buse 2H 0U es Pu At 0} ye
‘ue uy yes 200 ye a>vERUNvODP LONE Auy ‘OM IDOId OHI ue TRL Kp HPAP LL
‘ay 38 0 299 2 0 0430p hueduoD Buses} aI Sap gj 20 SBpS=DOW adUeG
‘P afens ea 20 wonsu s2ue99q hued) Bea) ahi 9:03.) $9 08 FERIOUANAP F #19609
<3 hurd Busey ox Jst09 jo abe ose? wy 9punaLBySuOREBIGO Jo Ave wy AYO
101s ued ue a “de ors vaomi9g dysusTea aM fo EUR awa) BEE
‘2 pur saworsn) uaamog @xseonear xn yo uoneuRush“sase> Bymoyoy au u =DNOU YNouM pe
‘sun Au esi st poe AuedwoD Bue mm esONES ALE BoUTUODND a pONRUR aH ©

"she an pus epuns epames Buena
“sh sauna uodn poses poxenoye>

['Annexure', 'Agreement']
['Annexure', 'Agreement']
['D26DC', 'D26DC Anhang 2 zu Anlage A.pdf', '/EMEAContracts/Contracts/Daimler - D26DC/2006 - Final Vertrag/D26DC Final komplett/D26DC Final Vertrag als pdf', nan, 'Annexure', 'Supportive Documents', 'Active', 'Found', 'pdf']
260
 

15:08 ers SS7i8L ‘CHG-MERIDIFN D.C

BESTELLEINTRITTSERKLARUNG CHG

 

ven 14072008
wieshon ws
Vow Peart Gosh, CHEMERDIAN \oostane group nb
‘Worsrererase #1 omar Laasng Austra Goi ox
Ueceroerrowirate 1
As420 on ‘oot Une
090 an

pero +450)81118 6060,

(vachstchend Miter genannt)

‘Ssh ochie Damen nd Haren,

esa gamulrsamoe Kunde vonage geupT GmbH (Mtr hat uns een Rahmanmiatvarrag abpo-
‘essen, Um im Ratren der enaener Besolonste Ono Abatsaraitarng Dvschen Thien und une 2
‘Saison, oan ie veoahmich ces Zacandokommons ones olspochancen Watcroles zu dan nachse-
‘Siren Gadngungon ne BexslongeninaaonHealar on. Oat Gorartvlunen dase Boni teoa-
‘oy bot 00,0000 EUR ond lt stan 31122008 ele, Mt cam Near got verona Arza

['Agreement', 'HPE Customer Terms - Portfolio']
['Agreement', 'HPE Customer Terms - Portfolio']
['D26DC', 'D26DC Anlage A.pdf', '/EMEAContracts/Contracts/Daimler - D26DC/2006 - Final Vertrag/D26DC Final komplett/D26DC Final Vertrag als pdf', nan, 'Agreement', 'Contract', 'Active', 'Found', 'pdf']
313
age
He wercestrg und Baring Ratan von Poe
‘be Ramonvereibag N 0260

 

 

Technology Solutions Group (86)
‘Consulting and Integration

zur Rahmenvoreinbarung Nr. 0250

LHP Werkleistungen und IT-Beratung lm Rahmen von Projekten

['Agreement']
['Agreement']
['D26DC', 'D26DC Anlage D.pdf', '/EMEAContracts/Contracts/Daimler - D26DC/2006 - Final Vertrag/D26DC Final komplett/D26DC Final Vertrag als pdf', nan, 'Agreement', 'Contract', 'Active', 'Found', 'pdf']
321
Papierfabrik Wattens GmbH & Co KG Austria T.S.P. Germany GmbH Germany OP papirna s.r.o. Czech Rep Dunafin Kft. Hungary 
[]
['JM4DF', 'JM4DF-affiliates list.xlsx', '/EMEAContracts/GWE South/GWE South/Austria - J/Standard - Custom/JM4DF 

[]
['TM2SR', 'Non standard clause approval form.xls', '/EMEAContracts/GWE South/GWE South/Austria - J/Standard - Custom/TM2SR - Swarovski', nan, '', '', 'Active', 'Unavailable', 'xls']
646
 

Daimler AG emer oy

tilt ol Senta
one | Commercial Set Suppon
iliac one zz. est ic
70546 Suapor/Gemony Ropu
See
28 Finer 2007

\Verlangerung vod Ergenzung der Rabatvereinbarungen zum
[Rahmenvertrog Nr. D26DC

See geet Her Stel,
swe bert voto per ern guichen micien wi ine mien, de der
‘Gatghiszstrows der in der eweiigen Aloge A. 8 urd C vereatorien abotlondiions
‘dr Rakmonerenbaring Nr 8260 ba sm 31. Desmbor 2007 verange

{der Anoge A: “Robathonionen & Bedingungen tir den Dithol

‘nH? Enterpae Conpuer Sytem Horvat, Sroge und agehaigen Sahwore
Frodsten haber sich ogee Andorngn geben

Frodstonloge 16001, 1EO02, 1£042 we 1€055 sind ausgelodneProduitnagen
[ich Tabele 1) he durch Podonogen dor Table 2 eset worden

‘Aaerdem wotde in dt Aaloge 8 di ode Bloc Adsts abel loin und
te haben ch dduch gene Andro

...

[]
['FM2DC', '[Untitled].pdf', '/EMEAContracts/France/FM2DC -- DCNS SA/New contract', 'DCNS SA', '', '', 'Active', 'Unavailable', 'pdf']
15915
 

DCN «a

CONTRAT EN MODE DIRECT ENTRE HP ET DCN

 

 

CONTRAT CADRE D'ACHAT DE PRODUITS INFORMATIQUES

 

Contrat N° G 06 78 909

 

 

Version 5-20 mars 2006

Entre

Hewitt Packard France, Société par Actions Simpliiées, au capital de 123 151 815 euros, dont le siége social
est stué 80 rue Camile Desmoulins £2130 Issy les Moulineaux, immatriculée au registre du commerce de
[Nantore sous le numéro B 652 031 857

 

‘Représentée par Monsieur Gilbert Farina, agissant en qualité de Directeur Général groupe systémes personnels
HP France, diment habit &Vetfet des présentes,

Chaprts désigné «HP»

Sune part
&

DCN, Societe Anonyme, au capital de 563.000 000 euros, dont ie sitge socal est situé au 2, rue Sextus Michel
‘4 Paris (75018), immatriculée au registre du commerce de Paris sous le numéro 441.133.808

RReprésentée par Monsieur Jean-Marc HEBRE

BNP PARIBAS
RENTAL SOLUTIONS

 

Arlus

 

A ORDER MANAGEMENT. VINCI@AP.COM
‘Sid: HEWLETT-PACKARD FRANCE

 

 

De: M Delatre Joan Wer

‘16: BNP Paribas Rental Solutions

 

Fav. al 0158 69 7070
ate 090472013 Fax: 01 $8 69 70 88
Nb pages): 2 Email :ean-yex delatrel@bnpparibas.com
Churgent

OBJET : Confirmation de financement

Notre référence : GPC980378-02/COMS39813/Vinel Energies

(référence & rappeler dans votre facture)
Référence Client : 102/IMHOFF

Messcus,

"Nos vous confnmons le inncement de I commande de notre client Vine! Energes pour wn montat de 7770 sous serve

‘ela ception des documents contacto

 

 

 

 

 

 

 

oe Dsignation Pri Unites HT [Prix Toul HT
| PonableBleBook 8570p Core 7™90e THE
i8-332014096Mo320G0115.67W7 Pros
1 3 Year Nbd Onsite wih Aesientl Damage Prtetion and ooo 00
Defective Media Retetio-HP
" iu Vi es

[Nous vous remercions ares Ia ctr

 

ARIUS BNP Paribas Rental Solations
‘Gestion des Factures Fourniseurs
mele Européen - Batineat A

4, rue Auguste 

[]
['FM3VN', '20130610122755830.pdf', '/EMEAContracts/France/FM3VN -- Vinci SA/Exceptional case CBN 810204480', 'Vinci SA', '', '', 'Active', 'Unavailable', 'pdf']
16035
BNP PARIBAS

 

 

 

RENTAL SOLUTIONS
Arius

“W_ ORDER: MANAGEMENT FINCK COM Dev M Det dean Yes

|S: HEWLETT-PACKARD FRANCE ‘SW: BNP Paribas Rental Solutions

Fee ra; 0158697070

Dave: 220172013 Fa 101 $569 7038

Nps) 2 Email: eam ves delatephnpparibas com
[vege

OBJET : Confirmation de financement
Notre référence : GPC980378-06/COMS76460/ Vinci Energies
(référence 4 rappeler dans votre facture)
Référence Client : 2013095-UF/TUNZINI

Messeus

‘ous vous confines ie Financement del commande de note cleat Vine! Energies pour un montant de 1.185,00 €, sous

reserve de a ception des documents contacts

 

 

 

 

 

 

 

 

 

 

Oe Disiguion Pec Unaaive et [Pei Taal HT
TT | elteDesk noo TW Core 5457003 2G I92MoS00GoSTIN 485.00 483,006
SoperMalt Optical DD/W-HP
1 (Cavior USM Standard Keyboards? ome ove
1 Sours Usb Optica

[]
['FM3VN', 'FM3VN VINCI non standard clause approval form.xls', '/EMEAContracts/France/FM3VN -- Vinci SA/Exceptional case CBN 810204480', 'Vinci SA', '', '', 'Active', 'Unavailable', 'xls']
16137
- - - - - - - - - - - - - - Périmètre VINCI France - - - - - - - - - - - - - - - - - - - - - - - - - - - Branche Pôle Division Direction Opérationnelle Dénomination sociale Nom de la Business Unit Adresse 1 Adresse 2 Adresse 3 Code postal Ville Pays SIREN  SIRET CONCESSIONS CONCESSIONS Aéroports SEAG (grenoble) SEAGI - Aéroport de Grenoble Saint-Geoirs - - 38590 ST ETIENNE DE ST GEOIRS FRANCE 450397047 00017 CONCESSIONS CONCESSIONS Aéroports SEAC (chambery) SEACA - Aéroport de Chambéry-Aix-les-Bains - - 73420 VIVIERS DU LAC FRANCE 453630642 00024 CONCESSIONS CONCESSIONS Aéroports SEACFA (clermont ferrand) SEACFA - Aéroport de Clermont-Ferrand Auvergne - - 63510 AULNAT FRANCE 499050615 00024 CONCESSIONS CONCESSIONS Aéroports SEAQC (Quimper Cornouaille) SEAQC - Aéroport de Quimper Cornouaille 

ProtocolError: <ConnectionTerminated error_code:ErrorCodes.COMPRESSION_ERROR, last_stream_id:1, additional_data:687061636b5f76616c75655f746f6f5f6c6f6e67>

In [38]:
r = pd.DataFrame(result,columns = ['PA No','File Name','Path','Customer Name','Record Type','Document Title','PA No Availability Status','Record Type Availability Status','File extension'])
r.style
fnft = r['Record Type Availability Status'] == "Found"
# fileerr = out[7] == "Unavailable"
fra = r[fnft]
fra.style

,PA No,File Name,Path,Customer Name,Record Type,Document Title,PA No Availability Status,Record Type Availability Status,File extension
2,JM3EN,CAM08.pdf,/EMEAContracts/GWE South/GWE South/Austria - J/Standard - Custom/JM3EN - Engel Austria GmbH,Engel Austria GmbH,Contract,Contract,Active,Found,pdf
3,JM3MG,JM3MG.pdf,/EMEAContracts/GWE South/GWE South/Austria - J/Standard - Custom/JM3MG - MAGNA STEYR Fahrzeugtechnik AG & Co KG,MAGNA STEYR Fahrzeugtechnik AG & Co KG,Contract,Contract,Active,Found,pdf
5,D26DC,D26DC Anhang 1 zu Anlage A.pdf,/EMEAContracts/Contracts/Daimler - D26DC/2006 - Final Vertrag/D26DC Final komplett/D26DC Final Vertrag als pdf,nan,Annexure,Supportive Documents,Active,Found,pdf
7,D26DC,D26DC Anhang 2 zu Anlage A.pdf,/EMEAContracts/Contracts/Daimler - D26DC/2006 - Final Vertrag/D26DC Final komplett/D26DC Final Vertrag als pdf,nan,Annexure,Supportive Documents,Active,Found,pdf
11,D26DC,D26DC Anlage A.pdf,/EMEAContracts/Contracts/Daimler - D26DC/2006 - Final Vertrag/D26DC Final komplett/D26DC Final Vertrag als pdf,nan,Agreement,Contract,Active,Found,pdf
12,D26DC,D26DC Anlage D.pdf,/EMEAContracts/Contracts/Daimler - D26DC/2006 - Final Vertrag/D26DC Final komplett/D26DC Final Vertrag als pdf,nan,Agreement,Contract,Active,Found,pdf
14,JM4DF,CAM08.pdf,/EMEAContracts/GWE South/GWE South/Austria - J/Standard - Custom/JM4DF - Delfortgroup AG/1st,Delfortgroup AG,Contract,Contract,Active,Found,pdf
15,JM4KH,CAM08.pdf,/EMEAContracts/GWE South/GWE South/Austria - J/Standard - Custom/JM4KH - Knoch Kern und Co. KG,Knoch Kern und Co. KG,Contract,Contract,Active,Found,pdf
18,D26DC,signed Renewal 2007.pdf,/EMEAContracts/Contracts/Daimler - D26DC/2007/Renewal 2007,nan,Extension,Supportive Documents,Active,Found,pdf
19,JMC01,EBA.pdf,/EMEAContracts/GWE South/GWE South/Austria - J/VSI/JMC01 - Cancom a+d IT-Solutions GmbH for Wiener Stadtische AG,nan,Contract,Contract,Active,Found,pdf


In [14]:
r.to_excel('iCertisPythonReportv2.xlsx')

In [33]:
textextract = "ez meh - peudes"
translator = Translator()
t = translator.translate(textextract)
transtext = t.text
transtext
# finalans = findrecord(transtext)
#                         record_availability = finalans[2]

'ez meh - peudes'

In [ ]:
print(oc)